In [4]:
import json
import pyproj
import os.path

In [5]:
geojson_directory = os.path.join('..', 'Data', 'geojson')
with open(os.path.join(geojson_directory, 'geojson_clean2.geojson')) as f:
    data = json.load(f)

In [6]:
# Define the source and target coordinate systems
src_crs = pyproj.CRS.from_epsg(2100)
target_crs = pyproj.CRS.from_epsg(4326)

# Create a transformer to convert coordinates
transformer = pyproj.Transformer.from_crs(src_crs, target_crs, always_xy=True)

# Convert the coordinates in the GeoJSON features
for feature in data['features']:
    geometry = feature['geometry']
    coordinates = geometry['coordinates']

    def convert_coordinates(coords):
        if isinstance(coords[0], list):
            # If it's a MultiPolygon, recursively convert each set of coordinates
            return [convert_coordinates(sub_coords) for sub_coords in coords]
        else:
            # Convert the coordinates
            x, y = coords
            lon, lat = transformer.transform(x, y)
            return [lon, lat]

    converted_coords = convert_coordinates(coordinates)
    geometry['coordinates'] = converted_coords

# Save the converted GeoJSON to a new file
with open(os.path.join(geojson_directory, 'output.geojson'), 'w') as f:
    json.dump(data, f)
